## World Bank Data Project

placeholder project description

step 1 
placeholder step description - load excel to df and start processing data

In [8]:
## start by reading and processing data
import pandas as pd

allSheets = pd.read_excel('data.xlsx', sheet_name=None)

sheetNames = list(allSheets.keys())
dataSheets = {name: allSheets[name] for name in sheetNames[1:]}

for name, df in dataSheets.items():
    print(name)
    print(df)

General
            Country Name Country Code  \
0                Albania          ALB   
1                 Angola          AGO   
2    Antigua and Barbuda          ATG   
3              Argentina          ARG   
4                Armenia          ARM   
..                   ...          ...   
156              Uruguay          URY   
157              Vanuatu          VUT   
158              Vietnam          VNM   
159   West Bank and Gaza          PSE   
160             Zimbabwe          ZWE   

                                           Agency Name         Currency  \
0                                      Bank of Albania      U.S. dollar   
1                             Banco Nacional de Angola   Angolan kwanza   
2         The Financial Services Regulatory Commission      U.S. dollar   
3                     Central Bank of Argentina (BCRA)   Argentine peso   
4                              Central Bank of Armenia    Armenian dram   
..                                               